# IF3170 Artificial Intelligence | Tugas Besar 2


Group Number: 69

Group Members:
- Owen Tobias Sinurat (13522131)
- Ahmad Thoriq Saputra (13522141)
- Muhammad Fatihul Irhab (13522143)
- Muhammad Fauzan Azhim (13522153)


## Import Libraries

In [4]:
import pandas as pd
import numpy as np
import scipy as scipy
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from functools import reduce
import pickle
from sklearn import datasets
import matplotlib.pyplot as plt
from collections import defaultdict
import os
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
from imblearn.over_sampling import RandomOverSampler

# Import other libraries if needed

ModuleNotFoundError: No module named 'sklearn'

## Import Dataset

In [43]:
additional_features_file = '1AmeA5tzBijKDgDbAo3s6bgXcPdpS8m0J'
basic_features_file = '1sZwaDBZobfYcF1YOKCk0bBM6TIVhuiJ0'
content_features_file = '1uk7WHpepvIOp16aUNVBEy0JX9t1crXzQ'
flow_features_file = '1eukApAaHYUWlXE-axWQWJSoauIEx33yE'
labels_file = '1eP0GODDpGkNYyywx4pQlfvHzieXfY55T'
time_features_file = '11ntLkX9qjYYyfA8xG790FjD2TXr0rSw9'
# UNSW_NB15_features_file = '1V292ulYQgH7sbSHXPbSgF9QZwK5DCujI'

additional_features_test_file = '1j5TBOwp7CeJv6AMaOITRjx41xmt51Wrz'
basic_features_test_file = '1zXQWmfMu12c90OYzcnn2En5jsm95QNLv'
content_features_test_file = '1KHiY0f4YfAlWRZ-FUY1jBpdP-fo1ab4T'
flow_features_test_file = '1MG5-tlaV2eQu4Nmm1FLiuSCxor_82LEi'
# labels_test_file = 'doesnt exists'
time_features_test_file = '1eP0GODDpGkNYyywx4pQlfvHzieXfY55T'
UNSW_NB15_features_test_file = '1_ppYX2lerIu852KzMjG3tXyu4qqzoFUd'

additional_features_df = pd.read_csv(f'https://drive.google.com/uc?id={additional_features_file}')
basic_features_df = pd.read_csv(f'https://drive.google.com/uc?id={basic_features_file}')
content_features_df = pd.read_csv(f'https://drive.google.com/uc?id={content_features_file}')
flow_features_df = pd.read_csv(f'https://drive.google.com/uc?id={flow_features_file}')
labels_features_df = pd.read_csv(f'https://drive.google.com/uc?id={labels_file}')
time_features_df = pd.read_csv(f'https://drive.google.com/uc?id={time_features_file}')
# UNSW_NB15_features_df = pd.read_csv(f'https://drive.google.com/uc?id={UNSW_NB15_features_file}')

additional_features_test = pd.read_csv(f'https://drive.google.com/uc?id={additional_features_test_file}')
basic_features_test = pd.read_csv(f'https://drive.google.com/uc?id={basic_features_test_file}')
content_features_test = pd.read_csv(f'https://drive.google.com/uc?id={content_features_test_file}')
flow_features_test = pd.read_csv(f'https://drive.google.com/uc?id={flow_features_test_file}')
# labels_test = pd.read_csv(f'https://drive.google.com/uc?id={labels_test_file}')
time_features_test = pd.read_csv(f'https://drive.google.com/uc?id={time_features_test_file}')
UNSW_NB15_features_test = pd.read_csv(f'https://drive.google.com/uc?id={UNSW_NB15_features_test_file}')

datasets = {
    'additional_features': additional_features_df,
    'basic_features': basic_features_df,
    'content_features': content_features_df,
    'flow_features': flow_features_df,
    'labels_features': labels_features_df,
    'time_features': time_features_df,
    # 'UNSW_NB15_features': UNSW_NB15_features_df
}
test_datasets = {
    'additional_features_test': additional_features_test,
    'basic_features_test': basic_features_test,
    'content_features_test': content_features_test,
    'flow_features_test': flow_features_test,
    # 'labels_test': labels_test,
    'time_features_test': time_features_test,
    'UNSW_NB15_features': UNSW_NB15_features_test
}

for key, dataset in datasets.items():
  dataset.head()

merged_dataset = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), datasets.values())
merged_dataset = merged_dataset.drop(columns=['label'])
merged_dataset.head()
for key, dataset in test_datasets.items():
  dataset.head()

,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,...,response_body_len,proto,attack_cat,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat
0,0.0,0.0,0.0,0.0,0.0,11.0,NaN,5.0,4.0,2.0,...,0.0,tcp,Normal,4449.110313,3234.831566,11.845558,6.261361,NaN,0.000444,0.000114
1,0.0,2.0,0.0,0.0,0.0,10.0,10.0,10.0,10.0,NaN,...,0.0,udp,Generic,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,NaN
2,0.0,NaN,0.0,0.0,0.0,4.0,4.0,2.0,2.0,1.0,...,0.0,tcp,Exploits,8561.040438,249.950547,165.386453,172.345750,0.158826,0.057902,0.100924
3,0.0,0.0,0.0,0.0,0.0,9.0,9.0,3.0,2.0,2.0,...,0.0,tcp,Normal,4053.086020,2918.730804,8.669644,4.496707,0.000558,0.000448,NaN
4,0.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,3.0,1.0,...,0.0,udp,Normal,0.000000,0.000000,0.008000,0.007000,0.000000,0.000000,0.000000


# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [3]:
for name, data in datasets.items():
  print("⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️")
  print(f"{name}")
  print("⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬇️⬇️⬇️⬇️⬇️⬇️")

  print(f"\n⚖️ Total record: {data.shape[0]}")

  missing_values = data.isnull().sum()
  print("\n🫙 Missing values per feature:")
  for feature, count in missing_values.items():
      print(f"{feature}: {count} missing values")

  print("\n🦄 Outliers per feature:")
  for column in data.select_dtypes(include=[np.number]).columns:
      data['z_score'] = scipy.stats.zscore(data[column].dropna())
      outliers = data[(np.abs(data['z_score']) > 3)]
      outlier_count = len(outliers)
      print(f"{column}: {outlier_count} outliers")

  print("\n")

⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️
additional_features
⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬇️⬇️⬇️⬇️⬇️⬇️

⚖️ Total record: 175341

🫙 Missing values per feature:
is_sm_ips_ports: 8746 missing values
ct_state_ttl: 8635 missing values
ct_flw_http_mthd: 8647 missing values
is_ftp_login: 8647 missing values
ct_ftp_cmd: 8842 missing values
ct_srv_src: 8851 missing values
ct_srv_dst: 8774 missing values
ct_dst_ltm: 8738 missing values
ct_src_ltm: 8823 missing values
ct_src_dport_ltm: 8775 missing values
ct_dst_sport_ltm: 8788 missing values
ct_dst_src_ltm: 8895 missing values
id: 0 missing values

🦄 Outliers per feature:
is_sm_ips_ports: 2632 outliers
ct_state_ttl: 1838 outliers
ct_flw_http_mthd: 1005 outliers
is_ftp_login: 2443 outliers
ct_ftp_cmd: 2443 outliers
ct_srv_src: 1560 outliers
ct_srv_dst: 1503 outliers
ct_dst_ltm: 5627 outliers
ct_src_ltm: 5621 outliers
ct_src_dport_ltm: 5871 outliers
ct_dst_sport_ltm: 595 outliers
ct_dst_src_ltm: 1538 outliers
id: 0 outliers


⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️
basic_features
⬆️⬆️⬆️⬆️

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [18]:
from sklearn.model_selection import train_test_split

X = merged_dataset.drop(['attack_cat'], axis=1)
y = merged_dataset['attack_cat']

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

y_train 169184     Generic
43790          DoS
169830     Generic
139375     Fuzzers
75510      Generic
            ...   
11331      Generic
158781    Exploits
127193    Exploits
114751    Analysis
159782     Generic
Name: attack_cat, Length: 140272, dtype: object


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [5]:
class MissingDataHandler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.numeric_imputer = {}
        self.categorical_imputer = {}

    def fit(self, X, y=None):
        categorical_features = [
            'proto',          # Transaction protocol
            'state',          # Protocol state
            'service'         # Service type
        ]

        numerical_features = [
            # Basic features
            'dur',              # Float
            'sbytes', 'dbytes', # Integer
            'sttl', 'dttl',    # Integer
            'sloss', 'dloss',  # Integer
            'sload', 'dload',  # Float
            'spkts', 'dpkts',  # Integer

            # Content features
            'swin', 'dwin',           # Integer
            'stcpb', 'dtcpb',        # Integer
            'smean', 'dmean',        # Integer
            'trans_depth',           # Integer
            'response_body_len',     # Integer

            # Time features
            'sjit', 'djit',          # Float
            'sinpkt', 'dinpkt',      # Float
            'tcprtt',               # Float
            'synack', 'ackdat',     # Float

            # Additional features
            'is_sm_ips_ports',      # Binary
            'ct_state_ttl',         # Integer
            'ct_flw_http_mthd',     # Integer
            'is_ftp_login',         # Binary
            'ct_ftp_cmd',           # Integer
            'ct_srv_src',           # Integer
            'ct_srv_dst',           # Integer
            'ct_dst_ltm',           # Integer
            'ct_src_ltm',           # Integer
            'ct_src_dport_ltm',     # Integer
            'ct_dst_sport_ltm',     # Integer
            'ct_dst_src_ltm'        # Integer
        ]

        for col in numerical_features:
            if col in X.columns:
                self.numeric_imputer[col] = X[col].median()

        for col in categorical_features:
            if col in X.columns:
                self.categorical_imputer[col] = X[col].mode()[0]
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for col, value in self.numeric_imputer.items():
            if col in X.columns:
                X[col].fillna(value, inplace=True)

        for col, value in self.categorical_imputer.items():
            if col in X.columns:
                X[col].fillna(value, inplace=True)
        return X

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [6]:
class OutlierHandler(BaseEstimator, TransformerMixin):
    def __init__(self, method='iqr', multiplier=1.5):
        self.method = method
        self.multiplier = multiplier
        self.bounds = {}

    def fit(self, X, y=None):
        numeric_cols = ['dur', 'sbytes', 'dbytes', 'sload', 'dload']

        for col in numeric_cols:
            if col in X.columns:
                Q1 = X[col].quantile(0.25)
                Q3 = X[col].quantile(0.75)
                IQR = Q3 - Q1
                self.bounds[col] = {
                    'lower': Q1 - self.multiplier * IQR,
                    'upper': Q3 + self.multiplier * IQR
                }
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for col, bounds in self.bounds.items():
            if col in X.columns:
                X[col] = np.clip(X[col], bounds['lower'], bounds['upper'])
        return X

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [7]:
class DuplicateHandler(BaseEstimator, TransformerMixin):
    def __init__(self, subset=None, keep='first'):
        self.subset = subset
        self.keep = keep

    def fit(self, X, y=None):  # Added y parameter
        return self

    def fit_transform(self, X, y=None):  # Added y parameter
        X = X.copy()
        if y is not None:
            unique_idx = ~X.duplicated(subset=self.subset, keep=self.keep)
            X_unique = X[unique_idx].reset_index(drop=True)
            y_unique = y[unique_idx].reset_index(drop=True)
            return X_unique, y_unique
        return X.drop_duplicates(subset=self.subset, keep=self.keep)

    def transform(self, X, y=None):  # Added y parameter
        X = X.copy()
        if y is not None:
            return X, y
        return X

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [8]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()

        if all(col in X.columns for col in ['sbytes', 'dbytes']):
            X['bytes_ratio'] = X['sbytes'] / (X['dbytes'] + 1)

        if all(col in X.columns for col in ['spkts', 'dpkts']):
            X['packet_ratio'] = X['spkts'] / (X['dpkts'] + 1)

        if all(col in X.columns for col in ['sttl', 'dttl']):
            X['ttl_diff'] = abs(X['sttl'] - X['dttl'])

        if all(col in X.columns for col in ['sload', 'dload']):
            X['load_ratio'] = X['sload'] / (X['dload'] + 1)

        return X

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [9]:
class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.numeric_columns = None

    def fit(self, X, y=None):
        all_numeric = ['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss',
                      'sload', 'dload', 'spkts', 'dpkts', 'sjit', 'djit', 'sinpkt',
                      'dinpkt', 'tcprtt', 'synack', 'ackdat']
        self.numeric_columns = [col for col in all_numeric if col in X.columns]
        self.scaler.fit(X[self.numeric_columns])
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X[self.numeric_columns] = self.scaler.transform(X[self.numeric_columns])
        return X

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [10]:
class FeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.label_encoders = {}
        self.categorical_columns = ['proto', 'state', 'service']
        self.target_encoder = LabelEncoder()

    def fit(self, X, y=None):
        for col in self.categorical_columns:
            if col in X.columns:
                self.label_encoders[col] = LabelEncoder()
                self.label_encoders[col].fit(X[col].astype(str))

        if y is not None:
            self.target_encoder.fit(y)
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for col, encoder in self.label_encoders.items():
            if col in X.columns:
                if 'unknown' not in encoder.classes_:
                  encoder.classes_ = np.append(encoder.classes_, 'unknown')
                X[col] = X[col].apply(lambda x: x if x in encoder.classes_ else 'unknown')
                X[col] = encoder.transform(X[col].astype(str))

        if y is not None:
            y_transformed = [
                self.target_encoder.transform([label])[0]
                if label in self.target_encoder.classes_ else -1
                for label in y
            ]
            return X, y_transformed
        return X

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [24]:
class ImbalanceHandler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.smote = SMOTE(random_state=42)

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if y is not None:
            if hasattr(self, '_is_fitted'):
                return X, y
            else:
                self._is_fitted = True
                X_resampled, y_resampled = self.smote.fit_resample(X, y)
                return X_resampled, y_resampled
        return X

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [12]:
class DimensionalityReducer(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=0.95):
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)

    def fit(self, X, y=None):
        self.pca.fit(X)
        return self

    def transform(self, X, y=None):
        X_reduced = self.pca.transform(X)
        return X_reduced, y if y is not None else X_reduced

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [44]:
from sklearn.pipeline import Pipeline

feature_pipeline = Pipeline([
    ('missing_handler', MissingDataHandler()),
    ('outlier_handler', OutlierHandler()),
    ('duplicate_handler', DuplicateHandler()),
    ('feature_engineer', FeatureEngineer()),
    ('feature_scaler', FeatureScaler()),
])

target_pipeline = Pipeline([
    ('feature_encoder', FeatureEncoder()),
    ('imbalance_handler', ImbalanceHandler()),
])

X_train_processed = feature_pipeline.fit_transform(X_train)
X_val_processed = feature_pipeline.transform(X_val)

# Initialize the transformers
encoder = FeatureEncoder()
imbalancer = ImbalanceHandler()
reducer = DimensionalityReducer()
# For training data
X_train_encoded = encoder.fit_transform(X_train_processed, y_train)
X_train_imbalanced, y_train_imbalanced = imbalancer.transform(X_train_encoded, y_train)
# X_train_reduced, y_train_reduced = reducer.fit_transform(X_train_processed, y_train)
# For validation data
X_val_encoded = encoder.transform(X_val_processed, y_val)
X_val_imbalanced, y_val_imbalanced = imbalancer.transform(X_val_encoded, y_val)
print('x-val',X_val_imbalanced)
print('y-val',X_val_imbalanced)
# X_val_reduced, y_val_reduced = reducer.transform(X_val_imbalanced, y_val_imbalanced)

<ipython-input-5-254aabd64c5d>:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(value, inplace=True)
<ipython-input-5-254aabd64c5d>:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({

x-val (        is_sm_ips_ports  ct_state_ttl  ct_flw_http_mthd  is_ftp_login  \
78281               0.0           1.0               0.0           0.0   
51971               1.0           2.0               0.0           0.0   
3117                0.0           1.0               1.0           0.0   
103442              0.0           2.0               0.0           0.0   
38743               0.0           2.0               0.0           0.0   
...                 ...           ...               ...           ...   
121868              0.0           2.0               0.0           0.0   
58535               0.0           1.0               0.0           0.0   
14023               0.0           2.0               0.0           0.0   
18489               0.0           1.0               0.0           0.0   
2216                0.0           2.0               0.0           0.0   

        ct_ftp_cmd  ct_srv_src  ct_srv_dst  ct_dst_ltm  ct_src_ltm  \
78281          0.0         1.0         2.0    

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [50]:
import numpy as np
from scipy.spatial.distance import cdist
from collections import Counter

class KNNClassifier:
    def __init__(self, k=3, metric="euclidean", weights="uniform", batch_size=1000):
        """
        Initialize the KNN Classifier.

        Parameters:
        -----------
        k : int, default=3
            Number of neighbors to use
        metric : str, default="euclidean"
            Distance metric to use
        weights : str, default="uniform"
            Weight function used in prediction
        batch_size : int, default=1000
            Number of test points to process in each batch
        """
        self.k = k
        self.metric = metric
        self.weights = weights
        self.batch_size = batch_size
        self.train_data = None
        self.train_labels = None

    def fit(self, train_data, train_labels):
        """
        Store the training data and labels.
        """
        self.train_data = np.asarray(train_data, dtype=np.float32)
        self.train_labels = np.asarray(train_labels)
        return self

    def _compute_batch_distances(self, test_batch):
        """
        Compute distances for a batch of test points.
        """
        if self.metric == 'euclidean':
            distances = cdist(test_batch, self.train_data, metric='euclidean')
        elif self.metric == 'manhattan':
            distances = cdist(test_batch, self.train_data, metric='cityblock')
        elif self.metric == 'minkowski':
            distances = cdist(test_batch, self.train_data, metric='minkowski')
        else:
            raise ValueError(f"Unsupported metric: {self.metric}")
        return distances

    def predict(self, test_points):
        """
        Predict labels for test points using batched processing.
        """
        # Validate that model has been trained
        if self.train_data is None:
            raise ValueError("Model has not been trained. Call 'fit' first.")


        # Prepare for batched prediction
        predictions = []

        # Process test points in batches
        for i in range(0, len(test_points), self.batch_size):
            # Get current batch
            test_batch = test_points[i:i+self.batch_size]

            # Compute distances for current batch
            distances = self._compute_batch_distances(test_batch)

            # Predict for each point in the batch
            batch_predictions = []
            for point_distances in distances:
                # Get indices of k nearest neighbors
                k_indices = np.argpartition(point_distances, self.k)[:self.k]
                print(f"k_indices: {k_indices}")

                # Get labels of k nearest neighbors
                k_labels = self.train_labels[k_indices]
                print(f"k_labels: {k_labels}")
                k_dist = point_distances[k_indices]
                print(f"k_dist: {k_dist}")

                # Apply weighting if specified
                if self.weights == 'distance':
                    # Avoid division by zero
                    weights = 1.0 / (k_dist + 1e-8)
                    unique_labels, _ = np.unique(k_labels, return_counts=True)
                    weighted_votes = []
                    for label in unique_labels:
                        label_mask = (k_labels == label)
                        weighted_vote = np.sum(weights[label_mask])
                        weighted_votes.append((label, weighted_vote))
                    prediction = max(weighted_votes, key=lambda x: x[1])[0]
                else:
                    # Uniform weighting (most common label)
                    prediction = Counter(k_labels).most_common(1)[0][0]

                batch_predictions.append(prediction)

            # Add batch predictions to overall predictions
            predictions.extend(batch_predictions)

        return np.array(predictions)

    def score(self, X_test, y_test):
        """
        Compute the accuracy of the classifier.
        """
        predictions = self.predict(X_test)
        return np.mean(predictions == y_test)

    def __repr__(self):
        return (f"MemoryEfficientKNNClassifier(k={self.k}, "
                f"metric='{self.metric}', "
                f"weights='{self.weights}', "
                f"batch_size={self.batch_size})")

In [ ]:
knn = KNNClassifier(k=3, metric="euclidean", weights="uniform", batch_size=1000)
knn.fit(X_train_imbalanced, y_train_imbalanced)
predictions = knn.predict(X_val_imbalanced)
accuracy = knn.score(X_val_imbalanced, y_val_imbalanced)
print(f"Accuracy: {accuracy}")


Found existing installation: numpy 1.23.0
Uninstalling numpy-1.23.0:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.10
    /usr/local/lib/python3.10/dist-packages/numpy-1.23.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-742d56dc.3.20.so
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.23.0
  Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.2 kB)
Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)


## B. Naive Bayes

In [31]:
class ImprovedNaiveBayes(BaseEstimator, ClassifierMixin):
    def __init__(self, smoothing=10**-8.5):
        self.smoothing = smoothing
        self.classes_ = None
        self.class_probabilities = {}
        self.feature_probabilities = {}
        self.class_counts = {}

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        self.classes_ = np.unique(y)
        n_samples, n_features = X.shape

        for cls in self.classes_:
            self.class_probabilities[cls] = np.sum(y == cls) / n_samples
            self.class_counts[cls] = np.sum(y == cls)

        self.feature_probabilities = {cls: [] for cls in self.classes_}
        for cls in self.classes_:
            X_cls = X[y == cls]
            for feature_idx in range(n_features):
                feature_vals = X_cls[:, feature_idx]
                unique_vals, counts = np.unique(feature_vals, return_counts=True)
                feature_prob = {
                    val: count / self.class_counts[cls]
                    for val, count in zip(unique_vals, counts)
                }

                self.feature_probabilities[cls].append(feature_prob)

        return self

    def predict(self, X):
        X = np.array(X)
        predictions = []
        for sample in X:
            class_scores = {}
            for cls in self.classes_:
                score = np.log(self.class_probabilities[cls])
                for feature_idx, feature_val in enumerate(sample):
                    feature_prob = self.feature_probabilities[cls][feature_idx].get(feature_val, self.smoothing)
                    score += np.log(feature_prob)
                class_scores[cls] = score
            predictions.append(max(class_scores, key=class_scores.get))

        return np.array(predictions)

    def get_params(self, deep=True):
        return {"smoothing": self.smoothing}

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

    def save_model(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
        print(f"Model saved in {filename}.")

    @staticmethod
    def load_model(filename):
        with open(filename, 'rb') as file:
            model = pickle.load(file)
        print(f"Model loaded from {filename}.")
        return model

    def submit(self, X, output_filename="predictions.csv"):
        predictions = self.predict(X)

        prediction_df = pd.DataFrame({
            'id': range(len(predictions)),
            'attack_cat': predictions
        })

        prediction_df.to_csv(output_filename, index=False)
        print(f"Predictions saved to {output_filename}.")

        return prediction_df

In [49]:
def evaluate_model(X_train, y_train, model, cv_folds=5):
    kfold = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cross_val_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

    print(f"\nCross-Validation Accuracy (Mean): {cross_val_scores.mean() * 100:.2f}%")
    print(f"Cross-Validation Accuracy (Standard Deviation): {cross_val_scores.std() * 100:.2f}%")

    return cross_val_scores.mean()

# X_train, X_test, y_train, y_test, scaler, label_encoders = preprocess_data(train_df)

nb = ImprovedNaiveBayes()
print('x train', X_train_imbalanced.info())
print('Y train', y_train_imbalanced.info())
nb.fit(X_train_imbalanced, y_train_imbalanced)

evaluate_model(X_train_imbalanced, y_train_imbalanced, nb, cv_folds=5)

predictions = nb.predict(X_val_imbalanced)

nb_sklearn = BernoulliNB()
nb_sklearn.fit(X_train_imbalanced, y_train_imbalanced)

evaluate_model(X_train_imbalanced, y_train_imbalanced, nb_sklearn, cv_folds=5)

predictions_sklearn = nb_sklearn.predict(X_val_imbalanced)

accuracy_custom = accuracy_score(y_val_imbalanced, predictions)
print(f"\nNaive Bayes kustom classification accuracy: {accuracy_custom * 100:.2f}%")
print("\nDetailed Classification Report (Custom Naive Bayes):")
print(classification_report(y_val_imbalanced, predictions))

accuracy_sklearn = accuracy_score(y_val_imbalanced, predictions_sklearn)
print(f"\nNaive Bayes from sklearn classification accuracy: {accuracy_sklearn * 100:.2f}%")
print("\nDetailed Classification Report (sklearn Naive Bayes):")
print(classification_report(y_val_imbalanced, predictions_sklearn))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448000 entries, 0 to 447999
Data columns (total 46 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   is_sm_ips_ports    448000 non-null  float64
 1   ct_state_ttl       448000 non-null  float64
 2   ct_flw_http_mthd   448000 non-null  float64
 3   is_ftp_login       448000 non-null  float64
 4   ct_ftp_cmd         448000 non-null  float64
 5   ct_srv_src         448000 non-null  float64
 6   ct_srv_dst         448000 non-null  float64
 7   ct_dst_ltm         448000 non-null  float64
 8   ct_src_ltm         448000 non-null  float64
 9   ct_src_dport_ltm   448000 non-null  float64
 10  ct_dst_sport_ltm   448000 non-null  float64
 11  ct_dst_src_ltm     448000 non-null  float64
 12  id                 448000 non-null  int64  
 13  state              448000 non-null  int64  
 14  dur                448000 non-null  float64
 15  sbytes             448000 non-null  float64
 16  db

KeyboardInterrupt: 

In [ ]:
loaded_model = ImprovedNaiveBayes.load_model('naive_bayes_model.pkl')
X_test_processed = preprocess_test_data(test_df, scaler, label_encoders)
prediction_df = loaded_model.submit(X_test_processed, output_filename="predictions.csv")

## C. ID3

In [45]:
class ID3(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, ccp_alpha=0.0):
        """Initialize the ID3 decision tree classifier."""
        self.tree = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.ccp_alpha = ccp_alpha

    def entropy(self, labels, base=np.e):
        """
        Compute the entropy of a list of labels.
        :param labels:
        :param base:
        :return:
        """
        labels = np.asarray(labels)
        _, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        return -np.sum(probabilities * np.log(probabilities) / np.log(base))

    def information_gain(self, X, y, feature):
        """
        Compute the information gain of a feature.
        :param X:
        :param y:
        :param feature:
        :return:
        """
        total_impurity = self.entropy(y)
        if isspmatrix(X):
            feature_series = pd.Series(X[:, feature].toarray().ravel())
        else:
            feature_series = X.iloc[:, feature]
        grouped = y.groupby(feature_series)
        weighted_impurity = sum((len(group) / len(y)) * self.entropy(group) for _, group in grouped)
        return total_impurity - weighted_impurity

    def best_feature_(self, X, y):
        """
        Find the best feature to split on.
        :param X:
        :param y:
        :return:
        """
        features = range(X.shape[1])
        gains = [self.information_gain(X, y, feature) for feature in features]
        if all(gain == 0 for gain in gains):
            return None  # No information gain
        return np.argmax(gains)

    def fit(self, X, y, features=None, depth=0):
        """
        Fit the decision tree classifier.
        :param X:
        :param y:
        :param features:
        :param depth:
        :return:
        """
        X, y = self._check_input(X, y)

        if features is None:
            features = list(range(X.shape[1]))

        # Stopping criteria
        if len(np.unique(y)) == 1:
            return {"value": y.iloc[0], "impurity": 0.0, "samples": len(y)}
        if len(features) == 0 or (self.max_depth is not None and depth >= self.max_depth):
            return {"value": y.mode()[0], "impurity": self.entropy(y), "samples": len(y)}
        if len(y) < self.min_samples_split or len(y) <= self.min_samples_leaf:
            return {"value": y.mode()[0], "impurity": self.entropy(y), "samples": len(y)}

        # Find the best feature and split data
        best_feature = self.best_feature_(X, y)
        if best_feature is None:
            return {"value": y.mode()[0], "impurity": self.entropy(y), "samples": len(y)}

        unique_vals = np.unique(X[:, best_feature].toarray() if isspmatrix(X) else X.iloc[:, best_feature])
        tree = {"feature": best_feature, "nodes": {}, "impurity": self.entropy(y), "samples": len(y)}

        def process_val(val):
            if isspmatrix(X):
                mask = (X[:, best_feature].toarray().ravel() == val)
            else:
                mask = (X.iloc[:, best_feature] == val).values
            subset_X = X[mask]
            subset_y = y[mask].reset_index(drop=True)

            if subset_y.empty:
                return val, {"value": None, "impurity": 0.0, "samples": 0}

            new_features = [f for f in features if f != best_feature]
            return val, self.fit(subset_X, subset_y, new_features, depth + 1)

        results = Parallel(n_jobs=-1)(delayed(process_val)(val) for val in unique_vals)

        for val, subtree in results:
            tree["nodes"][val] = subtree

        self.tree = tree
        return tree

    def prune(self, tree=None):
        """
        Prune the decision tree.
        :param tree:
        :return:
        """
        if tree is None:
            tree = self.tree

        if "nodes" not in tree or not tree["nodes"]:
            return tree["impurity"], None

        subtree_impurities = []
        subtree_trees = []
        for key, subtree in tree["nodes"].items():
            imp, subpruned = self.prune(subtree)
            subtree_impurities.append(imp)
            subtree_trees.append((key, subpruned))

        subtree_cost = sum(subtree_impurities) + self.ccp_alpha * len(tree["nodes"])
        if subtree_cost < tree["impurity"] + self.ccp_alpha:
            tree = {"value": max(Counter(
                [subtree["value"] for subtree in tree["nodes"].values() if "value" in subtree]).items(),
                                 key=lambda x: x[1])[0]}
        else:
            for key, subpruned in subtree_trees:
                if subpruned:
                    tree["nodes"][key] = subpruned

        return tree["impurity"], tree

    def predict_one(self, x, tree):
        """Predict a single data point."""
        if "value" in tree:
            return tree["value"]
        feature = tree["feature"]
        if x[feature] in tree["nodes"]:
            return self.predict_one(x, tree["nodes"][x[feature]])
        else:
            leaf_values = [subtree["value"] for subtree in tree["nodes"].values() if "value" in subtree]
            if leaf_values:
                return max(Counter(leaf_values).items(), key=lambda x: x[1])[0]
            return tree.get("value", None)

    def predict(self, X):
        """Predict multiple data points."""
        X = self._check_input(X)
        return np.array([self.predict_one(row, self.tree) for row in X.values])

    def _check_input(self, X, y=None):
        """Helper function to check and preprocess inputs."""
        if isspmatrix(X):
            X = csc_matrix(X)
        elif isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
        if isinstance(y, np.ndarray):
            y = pd.Series(y)
        return X, y

    def save_tree(self, filepath):
        """Save the trained decision tree to a file."""
        with open(filepath, 'wb') as f:
            pickle.dump(self.tree, f)

    def load_tree(self, filepath):
        """Load a decision tree from a file."""
        with open(filepath, 'rb') as f:
            self.tree = pickle.load(f)

    def evaluate(self, X, y):
        """Evaluate the model."""
        y_pred = self.predict(X)
        accuracy = accuracy_score(y, y_pred)
        classification_rep = classification_report(y, y_pred)
        f1_macro = f1_score(y, y_pred, average='macro')
        print(f'F1-Score Macro Average: {f1_macro}')
        print(f'Accuracy: {accuracy}')
        print(f'Classification Report:\n{classification_rep}')

    def submission(self, X, test_id=None):
        """Create a submission file."""
        y_pred = self.predict(X)
        submission = pd.DataFrame(y_pred, columns=['attack_cat'])
        if test_id is not None:
            submission.insert(0, "id", test_id)
        submission.to_csv('ID3-Save/submission_id3.csv', index=False)
        print("Submission file saved as 'submission_id3.csv'")

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`